In [1]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, log_loss
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
import os

In [4]:
vehicle = pd.read_csv("Cases/Vehicle_Silhouettes/Vehicle.csv")
vehicle

,Comp,Circ,D.Circ,Rad.Ra,Pr.Axis.Ra,Max.L.Ra,Scat.Ra,Elong,Pr.Axis.Rect,Max.L.Rect,Sc.Var.Maxis,Sc.Var.maxis,Ra.Gyr,Skew.Maxis,Skew.maxis,Kurt.maxis,Kurt.Maxis,Holl.Ra,Class
0,95,48,83,178,72,10,162,42,20,159,176,379,184,70,6,16,187,197,van
1,91,41,84,141,57,9,149,45,19,143,170,330,158,72,9,14,189,199,van
2,104,50,106,209,66,10,207,32,23,158,223,635,220,73,14,9,188,196,saab
3,93,41,82,159,63,9,144,46,19,143,160,309,127,63,6,10,199,207,van
4,85,44,70,205,103,52,149,45,19,144,241,325,188,127,9,11,180,183,bus
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
841,93,39,87,183,64,8,169,40,20,134,200,422,149,72,7,25,188,195,saab
842,89,46,84,163,66,11,159,43,20,159,173,368,176,72,1,20,186,197,van
843,106,54,101,222,67,12,222,30,25,173,228,721,200,70,3,4,187,201,saab
844,86,36,78,146,58,7,135,50,18,124,155,270,148,66,0,25,190,195,saab


In [5]:



X, y = vehicle.drop('Class', axis=1), vehicle['Class']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=25, stratify=y)

In [6]:
svm = SVC(kernel='linear')
svm.fit(X_train, y_train)
y_pred = svm.predict(X_test)
print(accuracy_score(y_test, y_pred))

0.8464566929133859


In [9]:
std_scaler = StandardScaler()
Cs = np.linspace(0.01, 5, 15)
dfs = ['ovo', 'ovr']
scores = []
for c in Cs:
    for s in dfs:
        svm = SVC(kernel='linear', C=c, decision_function_shape=s)
        pipe = Pipeline(steps = [ ('scaler', std_scaler), ('SVM', svm)])
        pipe.fit(X_train,y_train)
        y_pred = pipe.predict(X_test)
        scores.append([c,s, accuracy_score(y_test, y_pred)])

df_scores = pd.DataFrame(scores, columns=['C', 'decision', 'score'])
df_scores.sort_values('score', ascending=False)

,C,decision,score
25,4.287143,ovr,0.842520
24,4.287143,ovo,0.842520
16,2.861429,ovo,0.838583
17,2.861429,ovr,0.838583
8,1.435714,ovo,0.838583
9,1.435714,ovr,0.838583
23,3.930714,ovr,0.838583
22,3.930714,ovo,0.838583
14,2.505000,ovo,0.834646
15,2.505000,ovr,0.834646


In [10]:
std_scaler = StandardScaler()
Cs = np.linspace(0.01, 5, 15)
deg = [2,3,4]
dfs = ['ovo', 'ovr']
scores = []
for c in Cs:
    for s in dfs:
        for d in deg:
            svm = SVC(kernel='poly', C=c, degree=d, decision_function_shape=s)
            pipe = Pipeline(steps = [('scaler', std_scaler), ('SVM', svm)])
            pipe.fit(X_train,y_train)
            y_pred = pipe.predict(X_test)
            scores.append([c,s,d,accuracy_score(y_test, y_pred)])

df_scores = pd.DataFrame(scores, columns=['C', 'decision', 'degree', 'score'])
df_scores.sort_values('score', ascending=False)

,C,decision,degree,score
85,5.000000,ovo,3,0.791339
88,5.000000,ovr,3,0.791339
82,4.643571,ovr,3,0.783465
79,4.643571,ovo,3,0.783465
73,4.287143,ovo,3,0.767717
...,...,...,...,...
2,0.010000,ovo,4,0.381890
3,0.010000,ovr,2,0.377953
0,0.010000,ovo,2,0.377953
1,0.010000,ovo,3,0.346457


In [22]:
Cs = np.linspace(0.01, 5, 15)
gammas = np.linspace(0.01, 5, 15)
dfs = ['ovo', 'ovr']
scores = []
for g in tqdm(gammas):
    for c in Cs:
        for s in dfs:
            svm = SVC(kernel='rbf', C=c, gamma=g, decision_function_shape=s)
            pipe = Pipeline(steps = [('scaler', std_scaler), ('SVM', svm)])
            pipe.fit(X_train,y_train)
            y_pred = pipe.predict(X_test)
            scores.append([c, g, s, accuracy_score(y_test, y_pred)])

df_scores = pd.DataFrame(scores, columns=['C', 'gamma', 'dfs', 'score'])
df_scores.sort_values('score', ascending=False)

100%|██████████| 15/15 [00:04<00:00,  3.42it/s]


,C,gamma,dfs,score
58,5.000000,0.366429,ovo,0.707692
59,5.000000,0.366429,ovr,0.707692
71,1.792143,0.722857,ovr,0.707692
70,1.792143,0.722857,ovo,0.707692
9,1.435714,0.010000,ovr,0.707692
...,...,...,...,...
31,0.010000,0.366429,ovr,0.353846
391,0.010000,4.643571,ovr,0.353846
390,0.010000,4.643571,ovo,0.353846
420,0.010000,5.000000,ovo,0.353846


#### Glass Identification

In [15]:
glass = pd.read_csv('Cases/Glass_identification/Glass.csv')
glass

,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe,Type
0,1.52101,13.64,4.49,1.10,71.78,0.06,8.75,0.00,0.0,building_windows_float_processed
1,1.51761,13.89,3.60,1.36,72.73,0.48,7.83,0.00,0.0,building_windows_float_processed
2,1.51618,13.53,3.55,1.54,72.99,0.39,7.78,0.00,0.0,building_windows_float_processed
3,1.51766,13.21,3.69,1.29,72.61,0.57,8.22,0.00,0.0,building_windows_float_processed
4,1.51742,13.27,3.62,1.24,73.08,0.55,8.07,0.00,0.0,building_windows_float_processed
...,...,...,...,...,...,...,...,...,...,...
209,1.51623,14.14,0.00,2.88,72.61,0.08,9.18,1.06,0.0,headlamps
210,1.51685,14.92,0.00,1.99,73.06,0.00,8.40,1.59,0.0,headlamps
211,1.52065,14.36,0.00,2.02,73.42,0.00,8.44,1.64,0.0,headlamps
212,1.51651,14.38,0.00,1.94,73.61,0.00,8.48,1.57,0.0,headlamps


In [18]:

X, y = glass.drop('Type', axis=1), glass['Type']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=25, stratify=y)

In [19]:
svm = SVC(kernel='linear')
svm.fit(X_train, y_train)
y_pred = svm.predict(X_test)
print(accuracy_score(y_test, y_pred))

0.6461538461538462


In [20]:
std_scaler = StandardScaler()
Cs = np.linspace(0.01, 5, 15)
dfs = ['ovo', 'ovr']
scores = []
for c in Cs:
    for s in dfs:
        svm = SVC(kernel='linear', C=c, decision_function_shape=s)
        pipe = Pipeline(steps = [ ('scaler', std_scaler), ('SVM', svm)])
        pipe.fit(X_train,y_train)
        y_pred = pipe.predict(X_test)
        scores.append([c,s, accuracy_score(y_test, y_pred)])

df_scores = pd.DataFrame(scores, columns=['C', 'decision', 'score'])
df_scores.sort_values('score', ascending=False)

,C,decision,score
3,0.366429,ovr,0.615385
2,0.366429,ovo,0.615385
29,5.000000,ovr,0.615385
28,5.000000,ovo,0.615385
27,4.643571,ovr,0.615385
26,4.643571,ovo,0.615385
12,2.148571,ovo,0.600000
13,2.148571,ovr,0.600000
11,1.792143,ovr,0.600000
10,1.792143,ovo,0.600000


In [21]:
Cs = np.linspace(0.01, 5, 15)
gammas = np.linspace(0.01, 5, 15)
dfs = ['ovo', 'ovr']
scores = []
for g in tqdm(gammas):
    for c in Cs:
        for s in dfs:
            svm = SVC(kernel='rbf', C=c, gamma=g, decision_function_shape=s)
            pipe = Pipeline(steps = [('scaler', std_scaler), ('SVM', svm)])
            pipe.fit(X_train,y_train)
            y_pred = pipe.predict(X_test)
            scores.append([c, g, s, accuracy_score(y_test, y_pred)])

df_scores = pd.DataFrame(scores, columns=['C', 'degree', 'dfs', 'score'])
df_scores.sort_values('score', ascending=False)

100%|██████████| 15/15 [00:04<00:00,  3.26it/s]


,C,degree,dfs,score
58,5.000000,0.366429,ovo,0.707692
59,5.000000,0.366429,ovr,0.707692
71,1.792143,0.722857,ovr,0.707692
70,1.792143,0.722857,ovo,0.707692
9,1.435714,0.010000,ovr,0.707692
...,...,...,...,...
31,0.010000,0.366429,ovr,0.353846
391,0.010000,4.643571,ovr,0.353846
390,0.010000,4.643571,ovo,0.353846
420,0.010000,5.000000,ovo,0.353846


In [24]:
Cs = np.linspace(0.01, 5, 10)
gammas = np.linspace(0.01, 5, 10)
dfs = ['ovo', 'ovr']
scores = []
for g in tqdm(gammas):
    for c in Cs:
        for s in dfs:
            svm = SVC(kernel='rbf', C=c, gamma=g, decision_function_shape=s)
            pipe = Pipeline(steps = [('scaler', std_scaler), ('SVM', svm)])
            pipe.fit(X_train,y_train)
            y_pred = pipe.predict(X_test)
            scores.append([c, g, s, accuracy_score(y_test, y_pred)])

df_scores = pd.DataFrame(scores, columns=['C', 'gamma', 'dfs', 'score'])
df_scores.sort_values('score', ascending=False)

100%|██████████| 10/10 [00:01<00:00,  5.24it/s]


,C,gamma,dfs,score
6,1.673333,0.010000,ovo,0.707692
7,1.673333,0.010000,ovr,0.707692
4,1.118889,0.010000,ovo,0.692308
5,1.118889,0.010000,ovr,0.692308
27,1.673333,0.564444,ovr,0.692308
...,...,...,...,...
141,0.010000,3.891111,ovr,0.353846
161,0.010000,4.445556,ovr,0.353846
160,0.010000,4.445556,ovo,0.353846
180,0.010000,5.000000,ovo,0.353846
